In [1]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
# pretrained_model = "distilbert-base-uncased"
# pretrained_model = "xlm-roberta-large"
pretrained_model = "microsoft/deberta-v3-small" # not work
# pretrained_model = "kamalkraj/deberta-v2-xlarge"
# model_checkpoint = "/root/ner-model/test-ner/checkpoint-2500/"

source_dataset_name = "SpeedOfMagic/ontonotes_english"
checkpoint_name = 'checkpoint-14984'
model_checkpoint = f"../data/models/{pretrained_model}/{source_dataset_name}/{checkpoint_name}"

# use_pretrained_model = False
# if use_pretrained_model:
#     model_checkpoint = pretrained_model

batch_size = 16

dataset_name = "wnut_17"
# dataset_name = "SpeedOfMagic/ontonotes_english"

model_checkpoint

'../data/models/microsoft/deberta-v3-small/SpeedOfMagic/ontonotes_english/checkpoint-14984'

# Loading dataset

In [2]:
from datasets import load_dataset, load_metric

In [3]:
datasets = load_dataset(dataset_name)

Found cached dataset wnut_17 (/root/.cache/huggingface/datasets/wnut_17/wnut_17/1.0.0/077c7f08b8dbc800692e8c9186cdf3606d5849ab0e7be662e6135bb10eba54f9)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1287
    })
})

In [5]:
datasets["train"][0]

{'id': '0',
 'tokens': ['@paulwalk',
  'It',
  "'s",
  'the',
  'view',
  'from',
  'where',
  'I',
  "'m",
  'living',
  'for',
  'two',
  'weeks',
  '.',
  'Empire',
  'State',
  'Building',
  '=',
  'ESB',
  '.',
  'Pretty',
  'bad',
  'storm',
  'here',
  'last',
  'evening',
  '.'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  8,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [6]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(num_classes=13, names=['O', 'B-corporation', 'I-corporation', 'B-creative-work', 'I-creative-work', 'B-group', 'I-group', 'B-location', 'I-location', 'B-person', 'I-person', 'B-product', 'I-product'], id=None), length=-1, id=None)

In [7]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

In [8]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(datasets["train"])

,id,tokens,ner_tags
0,1785,"[New, boyz, have, come, a, long, way, in, the, game, and, still, going, strong, !, #muchlove]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,516,"[@weezywiii, yes, i, do, .., scouts, dont, smoke, week]","[O, O, O, O, O, O, O, O, O]"
2,600,"[Small, Biz, Tech, Tour, 2010, Launches, Five, City, Tour, MONTCLAIR, N.J., ...:, The, all, day, event, features, America's..., http://tinyurl.com/28hd9fu, #fb]","[O, O, O, O, O, O, O, O, O, B-location, B-location, O, O, O, O, O, O, O, O, O]"
3,2469,"[RT, @FitchRatings, :, New, SEC, rules, for, money, funds, may, affect, nearly, $500B, in, short-term, investments, by, corporations, https://t.co/UkkTFU41Sm]","[O, O, O, O, B-corporation, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
4,1378,"[RT, @ItsTheTeenLife, :, The, exact, day, that, I, wear, shorts, and, a, tee, shirt, everyone, shows, up, wearing, hoodies, and, jeans, ., -_-, #ItsTheTeenLife]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
5,1339,"[@kajelly_milk, (, :, but, hey, .., its, all, good, ., i, like, him, haha, &amp;omg, jason, kept, touching, me, today, D, ;, i, was, like, man, stop, touching, O, :]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
6,1189,"[RT, @ZodiacFacts, :, #ZodiacFacts, As, an, #Aries, romance, isn't, about, what, you're, after, --, it, 's, about, taking, chances, ,, exploring, and, keeping, an, ...]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
7,644,"[My, first, assignment, was, a, medical, illustration, job, through, a, friend, ., After, meeting, with, the, surgeon, ,, I, comple, ...]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
8,1988,"[Click, 4, Details, &gt;, &gt;, &gt;, http://schmap.it/u046mn, SEDUCTION, SATURDAYS, @, CALABASH, LOUNGE, SEP, 25TH, #DABOSS, #CALABASH, #WJE]","[O, O, O, O, O, O, O, O, O, O, B-location, I-location, O, O, O, O, O]"
9,563,"[@JoxVox, Just, stressing, about, the, fitted, ones, though, -, opro, were, in, school, today, and, I, had, forgotten, to, register, him, for, fitting, ..., whoops]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"


# Preprocessing dataset

In [10]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [11]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [12]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [1, 5365, 261, 291, 269, 311, 4378, 300, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
example = datasets["train"][4]
print(example["tokens"])

['4Dbling', "'s", 'place', 'til', 'monday', ',', 'party', 'party', 'party', '.', '&lt;', '3']


In [14]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', '▁4', 'D', 'bling', "▁'", 's', '▁place', '▁til', '▁monday', '▁,', '▁party', '▁party', '▁party', '▁.', '▁&', 'lt', ';', '▁3', '[SEP]']


In [15]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(12, 19)

In [16]:
print(tokenized_input.word_ids())

[None, 0, 0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 11, None]


In [17]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

19 19


In [18]:
label_all_tokens = True

In [19]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [20]:
tokenize_and_align_labels(datasets['train'][:5])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[1, 1944, 44086, 17328, 325, 382, 268, 262, 866, 292, 399, 273, 382, 358, 821, 270, 375, 1033, 323, 6251, 867, 4098, 1842, 74224, 323, 9452, 966, 4084, 422, 437, 1882, 323, 2], [1, 1007, 2036, 1864, 26742, 877, 22613, 14706, 9149, 5983, 270, 14538, 3005, 264, 4024, 323, 456, 1871, 294, 320, 320, 94698, 260, 549, 320, 2823, 452, 60317, 4921, 2], [1, 916, 43475, 39750, 2482, 960, 9591, 12852, 8794, 265, 1499, 1524, 323, 260, 260, 1871, 294, 320, 320, 5435, 260, 701, 320, 45036, 38842, 3206, 693, 953, 57483, 2], [1, 561, 269, 312, 437, 406, 288, 262, 832, 323, 2], [1, 453, 691, 27020, 382, 268, 470, 17846, 44840, 366, 915, 915, 915, 323, 429, 19011, 346, 404, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0

In [21]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/wnut_17/wnut_17/1.0.0/077c7f08b8dbc800692e8c9186cdf3606d5849ab0e7be662e6135bb10eba54f9/cache-4854bfd9b03b0a3e.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/wnut_17/wnut_17/1.0.0/077c7f08b8dbc800692e8c9186cdf3606d5849ab0e7be662e6135bb10eba54f9/cache-6b12975f31dd36c1.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/wnut_17/wnut_17/1.0.0/077c7f08b8dbc800692e8c9186cdf3606d5849ab0e7be662e6135bb10eba54f9/cache-a7e39f9c18e179f0.arrow


# Fine-tuning the model

In [22]:
from transformers import AutoConfig, AutoModelForTokenClassification, TrainingArguments, Trainer
# config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list), ignore_mismatched_sizes=True) #, from_tf=True

2022-10-07 05:54:35.151444: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-07 05:54:35.257302: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-07 05:54:35.619787: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-07 05:54:35.619832: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [33]:
args = TrainingArguments(
    # f"test-{task}",
    f"../data/models/{pretrained_model}/{dataset_name}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=8,
    weight_decay=0.01,
    
    save_strategy='epoch',
    save_total_limit = 3
    
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [34]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [35]:
metric = load_metric("seqeval")
# from evaluate import evaluator

# task_evaluator = evaluator("token-classification")

In [36]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'person': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [37]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [38]:
import torch
torch.cuda.empty_cache()

In [39]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [40]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: tokens, ner_tags, id. If tokens, ner_tags, id are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.
/root/.cache/pypoetry/virtualenvs/sequence-labeling-1mH2o5_I-py3.8/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3394
  Num Epochs = 8
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1704
/root/.cache/pypoetry/virtualenvs/sequence-labeling-1mH2o5_I-py3.8/lib/python3.8/site-package

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.284694,0.694725,0.544894,0.610755,0.949000
2,No log,0.269112,0.658754,0.586268,0.620401,0.951731
3,0.025500,0.291965,0.689402,0.578345,0.629009,0.952384
4,0.025500,0.308104,0.675818,0.563380,0.614498,0.950009
5,0.013400,0.315007,0.681296,0.573944,0.623029,0.951671
6,0.013400,0.326134,0.690372,0.555458,0.615610,0.951137
7,0.013400,0.345991,0.689113,0.540493,0.605821,0.950365
8,0.008600,0.341914,0.688596,0.552817,0.613281,0.950899


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: tokens, ner_tags, id. If tokens, ner_tags, id are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1009
  Batch size = 16
Saving model checkpoint to ../data/models/microsoft/deberta-v3-small/wnut_17/checkpoint-213
Configuration saved in ../data/models/microsoft/deberta-v3-small/wnut_17/checkpoint-213/config.json
Model weights saved in ../data/models/microsoft/deberta-v3-small/wnut_17/checkpoint-213/pytorch_model.bin
tokenizer config file saved in ../data/models/microsoft/deberta-v3-small/wnut_17/checkpoint-213/tokenizer_config.json
Special tokens file saved in ../data/models/microsoft/deberta-v3-small/wnut_17/checkpoint-213/special_tokens_map.json
Deleting older checkpoint [../data/models/microsoft/deberta-v3-small/wnut_17/checkpoint-426] due to a

TrainOutput(global_step=1704, training_loss=0.01470329867842052, metrics={'train_runtime': 98.9625, 'train_samples_per_second': 274.367, 'train_steps_per_second': 17.219, 'total_flos': 355941845237568.0, 'train_loss': 0.01470329867842052, 'epoch': 8.0})

In [41]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: tokens, ner_tags, id. If tokens, ner_tags, id are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1009
  Batch size = 16
/root/.cache/pypoetry/virtualenvs/sequence-labeling-1mH2o5_I-py3.8/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:745: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_scores = torch.bmm(query_layer, key_layer.transpose(-1, -2)) / torch.tensor(
/root/.cache/pypoetry/virtualenvs/sequence-labeling-1mH2o5_I-py3.8/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:829: UserWarning: To copy construct from a tens

{'eval_loss': 0.34191426634788513,
 'eval_precision': 0.6885964912280702,
 'eval_recall': 0.5528169014084507,
 'eval_f1': 0.61328125,
 'eval_accuracy': 0.9508994834649409,
 'eval_runtime': 0.9235,
 'eval_samples_per_second': 1092.603,
 'eval_steps_per_second': 69.303,
 'epoch': 8.0}

In [42]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: tokens, ner_tags, id. If tokens, ner_tags, id are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1009
  Batch size = 16


{'corporation': {'precision': 0.34615384615384615,
  'recall': 0.3829787234042553,
  'f1': 0.36363636363636365,
  'number': 47},
 'creative-work': {'precision': 0.475,
  'recall': 0.26573426573426573,
  'f1': 0.3408071748878923,
  'number': 143},
 'group': {'precision': 0.2345679012345679,
  'recall': 0.34545454545454546,
  'f1': 0.27941176470588236,
  'number': 55},
 'location': {'precision': 0.7195121951219512,
  'recall': 0.6145833333333334,
  'f1': 0.6629213483146068,
  'number': 96},
 'person': {'precision': 0.8374291115311909,
  'recall': 0.6868217054263566,
  'f1': 0.7546848381601362,
  'number': 645},
 'product': {'precision': 0.5795454545454546,
  'recall': 0.34,
  'f1': 0.4285714285714286,
  'number': 150},
 'overall_precision': 0.6885964912280702,
 'overall_recall': 0.5528169014084507,
 'overall_f1': 0.61328125,
 'overall_accuracy': 0.9508994834649409}